In [1]:
# PymuPDFライブラリインストール(import fitz)
# !pip install pymupdf

In [1]:
import time
import pandas as pd
import numpy as np
import extract_images
import extract_figure

[NbConvertApp] Converting notebook extract_images.ipynb to python
[NbConvertApp] Writing 2785 bytes to extract_images.py


In [2]:
def merge_image_and_figure_info(input_folder, image_csv_filename, output_root_folder, pattern, figure_csv_filename, output_csv):
    """
    画像情報と図番号情報をマージしてCSVファイルに保存する関数。

    Args:
        input_folder (str): PDFファイルが格納されているフォルダのパス。
        image_csv_filename (str): 画像情報出力CSVファイルのパス。
        output_root_folder (str): 画像を保存するルートフォルダへのパス。
        pattern (str): 図番号の正規表現パターン。
        figure_csv_filename (str): 出力CSVファイルのパス。
        output_csv (str): 出力CSVファイルのパス。
    """
    
    before = time.time()
    
    # 画像情報と図番号情報を含むデータフレームを作成する
    image_info = extract_images.extract_images(input_folder, image_csv_filename, output_root_folder) # 画像情報を取得する関数
    figure_info = extract_figure.extract_figure(input_folder, pattern, figure_csv_filename)  # 図番号情報を取得する関数

    # マージしたデータを格納するリストを初期化
    merged_data = []

    # 画像の座標を取得
    for _, img_row in image_info.iterrows():
        img_page = img_row['ページ番号']
        img_x0, img_y0, img_x1, img_y1 = img_row['x0'], img_row['y0'], img_row['x1'], img_row['y1']

        # 一致する図番号情報をフィルタリング
        matching_figures = figure_info[
            (figure_info['PDFファイル'] == img_row['PDFファイル']) &
            (figure_info['ページ番号'] == img_page)
        ]

        if not matching_figures.empty:
            best_match = None
            min_distance = np.inf

            # 図番号座標を取得
            for _, fig_row in matching_figures.iterrows():
                fig_x0, fig_y0, fig_x1, fig_y1 = fig_row['x0'], fig_row['y0'], fig_row['x1'], fig_row['y1']

                # 画像と図番号の座標の差を計算(ユークリッド距離)
                distance = np.sqrt((img_x0 - fig_x0)**2 + (img_y0 - fig_y0)**2)
                

                # 座標の差が小さい方を選択
                if distance < min_distance:
                    best_match = fig_row
                    min_distance = distance

            if best_match is not None:
                # 図番号と画像の座標差をデータフレームに保存
                x0_diff = img_x0 - best_match['x0']
                y0_diff = img_y0 - best_match['y0']
                x1_diff = img_x1 - best_match['x1']
                y1_diff = img_y1 - best_match['y1']

                merged_data.append({
                    'PDFファイル': img_row['PDFファイル'],
                    'ページ番号': img_page,
                    '画像ファイル': img_row['画像ファイル'],
                    '図番号': best_match['図番号'],
                    'x0_diff': f'{x0_diff:.5f}',
                    'y0_diff': f'{y0_diff:.5f}',
                    'x1_diff': f'{x1_diff:.5f}',
                    'y1_diff': f'{y1_diff:.5f}'
                })

    # マージしたデータをデータフレームに変換
    merged_df = pd.DataFrame(merged_data)

    # マージしたデータフレームを保存
    merged_df.to_csv(output_csv, index=False)
    
    after = time.time()
    time_diff = after - before
    print(f"It tooks {time_diff} sec.")


In [3]:
# 使用例:
input_folder = '../sample-pdf'  # PDFファイルが格納されているフォルダのパス
image_csv_filename = 'image_info.csv' # 画像情報出力CSVファイルのパス。
output_root_folder = "output-images" # 画像を保存するルートフォルダへのパス。
# pattern = r"【図 [0-9]+\-[0-9]+】"  # 図番号の正規表現パターン
pattern = r"【第[0-9]+\-[0-9]+\-[0-9]+】"
figure_csv_filename = 'figure_info.csv' # 図番号情報出力CSVファイルのパス。
output_csv = 'merged_info.csv'  # 出力CSVファイルのパス
merge_image_and_figure_info(input_folder, image_csv_filename, output_root_folder, pattern, figure_csv_filename, output_csv)


Matched text: 【第111-1-1】 at coordinates: (37.29970169067383, 64.42608642578125), (47.29970169067383, 77.45809173583984)
Matched text: 【第111-2-1】 at coordinates: (107.38629913330078, 512.6466064453125), (117.38629913330078, 525.6785888671875)
Matched text: 【第111-2-2】 at coordinates: (302.976806640625, 64.42608642578125), (312.976806640625, 77.45809173583984)
Matched text: 【第111-2-3】 at coordinates: (40.29869842529297, 64.42608642578125), (50.29869842529297, 77.45809173583984)
Matched text: 【第111-3-1】 at coordinates: (302.976806640625, 374.1741027832031), (312.976806640625, 387.20611572265625)
Matched text: 【第111-3-2】 at coordinates: (37.29970169067383, 594.851318359375), (47.29970169067383, 607.88330078125)
Matched text: 【第111-4-3】 at coordinates: (40.29869842529297, 400.4261169433594), (50.29869842529297, 413.4581298828125)
Matched text: 【第111-4-4】 at coordinates: (305.97589111328125, 64.42608642578125), (315.97589111328125, 88.4560775756836)
Matched text: 【第111-4-5】 at coordinates: (3